# This program uses model.pkl to predict resale prices based on the provided information.

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import numpy as np
import joblib  # For loading the model
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime, timedelta

# Load the HDB resale dataset (for dropdown values)
df = pd.read_csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')  # Adjust the path

# Load the trained model
model = joblib.load('model.pkl')  # Adjust the path to your model file

# Initialize the Tkinter window
root = tk.Tk()
root.title("HDB Resale Price Prediction")
root.geometry("600x700")

# Helper functions
def get_town_options():
    """Return distinct towns from the dataset."""
    return sorted(df['town'].unique().tolist())

def get_flat_model_options():
    """Return distinct flat models from the dataset."""
    return sorted(df['flat_model'].unique().tolist())

def get_storey_range_options():
    """Return distinct storey ranges from the dataset."""
    return sorted(df['storey_range'].unique().tolist())

def get_flat_type_options():
    """Return distinct flat types from the dataset."""
    return sorted(df['flat_type'].unique().tolist())

def get_month_time_options():
    """Return the next 12 months as a dropdown with Unix timestamps."""
    current_date = datetime.now()
    return [
        (current_date + timedelta(days=30 * i)).strftime("%Y-%m-01")
        for i in range(12)
    ]

# Pre-fit the encoder with unique values from the dataset
encoder_town = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder_flat_model = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoders on the distinct values
encoder_town.fit(df[['town']])
encoder_flat_model.fit(df[['flat_model']])

def predict_resale_price():
    try:
        # Retrieve user inputs
        floor_area_sqm = float(floor_area_var.get())
        lease_commence_date = int(lease_date_var.get())
        remaining_lease_year = float(remaining_lease_var.get())
        storey_range = storey_var.get()
        storey_value = (int(storey_range[:2]) // 3) + 1
        flat_type = flat_type_var.get()
        flat_type_n = {
            '1 ROOM': 1, '2 ROOM': 2, '3 ROOM': 3,
            '4 ROOM': 4, '5 ROOM': 5
        }.get(flat_type, 6)
        month_time = month_time_var.get()
        month_time_unix = int(datetime.strptime(month_time, "%Y-%m-01").timestamp() * 1e9)
        town = town_var.get()
        flat_model = flat_model_var.get()

        # One-hot encode 'town' and 'flat_model'
        town_encoded = encoder_town.transform([[town]])
        flat_model_encoded = encoder_flat_model.transform([[flat_model]])

        # Combine all features into a single array
        input_features = np.array([
            floor_area_sqm, lease_commence_date, remaining_lease_year,
            storey_value, flat_type_n, month_time_unix
        ])
        input_features = np.concatenate([input_features, town_encoded[0], flat_model_encoded[0]])

        # Display the input features for debugging
        # messagebox.showinfo("Debug Info", f"Input Features: {input_features}")

        # Reshape for model prediction
        input_features = input_features.reshape(1, -1)

        # Predict resale price
        predicted_price = model.predict(input_features)[0]
        messagebox.showinfo("Prediction Result", f"Predicted Resale Price: ${predicted_price:.2f}")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")


# Layout widgets for user input
tk.Label(root, text="Enter the following details for prediction:", font=("Arial", 14)).pack(pady=10)

# Floor Area
floor_area_frame = tk.Frame(root)
floor_area_frame.pack(pady=5)
tk.Label(floor_area_frame, text="Floor Area (sqm):").grid(row=0, column=0, padx=5)
floor_area_var = tk.StringVar()
tk.Entry(floor_area_frame, textvariable=floor_area_var).grid(row=0, column=1)

# Lease Commencement Date
lease_date_frame = tk.Frame(root)
lease_date_frame.pack(pady=5)
tk.Label(lease_date_frame, text="Lease Commencement Date:").grid(row=0, column=0, padx=5)
lease_date_var = tk.StringVar()
tk.Entry(lease_date_frame, textvariable=lease_date_var).grid(row=0, column=1)

# Remaining Lease (Years)
remaining_lease_frame = tk.Frame(root)
remaining_lease_frame.pack(pady=5)
tk.Label(remaining_lease_frame, text="Remaining Lease (Years):").grid(row=0, column=0, padx=5)
remaining_lease_var = tk.StringVar()
tk.Entry(remaining_lease_frame, textvariable=remaining_lease_var).grid(row=0, column=1)

# Storey Range Dropdown
storey_frame = tk.Frame(root)
storey_frame.pack(pady=5)
tk.Label(storey_frame, text="Floor level:").grid(row=0, column=0, padx=5)
storey_var = tk.StringVar()
storey_dropdown = ttk.Combobox(storey_frame, textvariable=storey_var, values=get_storey_range_options())
storey_dropdown.grid(row=0, column=1)

# Flat Type Dropdown
flat_type_frame = tk.Frame(root)
flat_type_frame.pack(pady=5)
tk.Label(flat_type_frame, text="Flat Type:").grid(row=0, column=0, padx=5)
flat_type_var = tk.StringVar()
flat_type_dropdown = ttk.Combobox(flat_type_frame, textvariable=flat_type_var, values=get_flat_type_options())
flat_type_dropdown.grid(row=0, column=1)

# Month Time Dropdown
month_time_frame = tk.Frame(root)
month_time_frame.pack(pady=5)
tk.Label(month_time_frame, text="Sell Month:").grid(row=0, column=0, padx=5)
month_time_var = tk.StringVar()
month_time_dropdown = ttk.Combobox(month_time_frame, textvariable=month_time_var, values=get_month_time_options())
month_time_dropdown.grid(row=0, column=1)

# Town Dropdown
town_frame = tk.Frame(root)
town_frame.pack(pady=5)
tk.Label(town_frame, text="Town:").grid(row=0, column=0, padx=5)
town_var = tk.StringVar()
town_dropdown = ttk.Combobox(town_frame, textvariable=town_var, values=get_town_options())
town_dropdown.grid(row=0, column=1)

# Flat Model Dropdown
flat_model_frame = tk.Frame(root)
flat_model_frame.pack(pady=5)
tk.Label(flat_model_frame, text="Flat Model:").grid(row=0, column=0, padx=5)
flat_model_var = tk.StringVar()
flat_model_dropdown = ttk.Combobox(flat_model_frame, textvariable=flat_model_var, values=get_flat_model_options())
flat_model_dropdown.grid(row=0, column=1)

# Predict Button
tk.Button(root, text="Predict Resale Price", command=predict_resale_price).pack(pady=20)

# Run the application
root.mainloop()


C:\Users\Razer\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\Razer\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
